<h1>EDA to get buyer and seller valuation</h1>

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [48]:
# Load excel file into a DataFrame
df = pd.read_excel("../datasets/resalepricesadjusted.xlsx")

df.head()
df['Neighborhood score'] = df['Neighborhood score'].astype(float)
df['flat_type'] = df['flat_type'].astype(float)
df['remaining_lease'] = df['remaining_lease'].astype(float)
df['resale_price'] = df['resale_price'].astype(float)
df['mean price'] = df['mean price'].astype(float)
df['ln_resale_price'] = df['ln_resale_price'].astype(float)
df['ln_mean_price'] = df['ln_mean_price'].astype(float)

In [61]:
# Group by 'neighborhood score' and calculate the average resale price
neighborhood_avg_resale_price = df.groupby('month')['resale_price'].transform('mean')

# Create a new column 'average_resale_price' to store the average resale price for each neighborhood score
df['average_resale_price'] = neighborhood_avg_resale_price

print('after')
print(df.head())

after
     month        town  Neighborhood score  flat_type  lease_commence_date  \
0  2017-01  ANG MO KIO                 4.0        2.0                 1979   
1  2017-01  ANG MO KIO                 4.0        3.0                 1978   
2  2017-01  ANG MO KIO                 4.0        3.0                 1980   
3  2017-01  ANG MO KIO                 4.0        3.0                 1980   
4  2017-01  ANG MO KIO                 4.0        3.0                 1980   

   remaining_lease  resale_price  mean price  ln_resale_price  ln_mean_price  \
0             61.0      232000.0   413971.21            12.35      12.933552   
1             60.0      250000.0   413971.21            12.43      12.933552   
2             62.0      262000.0   413971.21            12.48      12.933552   
3             62.0      265000.0   413971.21            12.49      12.933552   
4             62.0      265000.0   413971.21            12.49      12.933552   

   average_resale_price  
0          427506.

In [62]:
print(df.tail())

          month    town  Neighborhood score  flat_type  lease_commence_date  \
175393  2024-03  YISHUN                 5.0        6.0                 1987   
175394  2024-03  YISHUN                 5.0        6.0                 1988   
175395  2024-03  YISHUN                 5.0        6.0                 1988   
175396  2024-03  YISHUN                 5.0        6.0                 1988   
175397  2024-03  YISHUN                 5.0        6.0                 1987   

        remaining_lease  resale_price  mean price  ln_resale_price  \
175393             62.0      851888.0   532994.77            13.66   
175394             63.0      830000.0   532994.77            13.63   
175395             63.0      820000.0   532994.77            13.62   
175396             63.0      860000.0   532994.77            13.66   
175397             62.0      780000.0   532994.77            13.57   

        ln_mean_price  average_resale_price  
175393      13.186267         591285.836329  
175394      

In [50]:
# Assuming df is your DataFrame
# Selecting only the specified columns from the dataframe
X_columns = ['average_resale_price', 'Neighborhood score', 'flat_type', 'remaining_lease']
X = df[X_columns]

# Find rows with NaN values in the specified columns
rows_with_nan = X[X.isna().any(axis=1)]

# Display rows with NaN values
print(rows_with_nan)

Empty DataFrame
Columns: [average_resale_price, Neighborhood score, flat_type, remaining_lease]
Index: []


In [51]:
X_columns = ['average_resale_price','Neighborhood score','flat_type','remaining_lease']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [52]:
# Making predictions
y_pred = model.predict(X_test)

In [53]:
# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coefficients)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.04843120405599286
R-squared: 0.5689407393874871

Coefficients:
                      Coefficient
average_resale_price     0.902600
Neighborhood score       0.013297
flat_type                0.235408
remaining_lease          0.006035

Intercept: -0.26345270437825263


<h1>Formula</h1>

$$
\text{resale\_price} = e^{(0.902 \times \ln(\text{mean\_price})) + (0.013 \times \text{Neighborhood score}) +
 (0.235 \times \text{number\_of\_rooms}) + (0.006 \times \text{remaining\_lease}) - 0.263}
$$

<h1>Seller Valuation</h1>

In [54]:
X_columns = ['average_resale_price', 'remaining_lease']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [55]:
y_pred = model.predict(X_test)

In [56]:
# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coefficients)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.09173942731234888
R-squared: 0.18347828683843803

Coefficients:
                      Coefficient
average_resale_price     0.946772
remaining_lease          0.008878

Intercept: -0.014472321274512367


<h1>Formula</h1>

$$
\text{resale\_price} = e^{(0.946 \times \ln(\text{mean\_price}) + 0.008 \times \text{remaining\_lease}) - 0.0144}
$$